import package

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
!pip install bioinfokit
from bioinfokit.visuz import cluster
from sklearn.manifold import TSNE
from keras.datasets import mnist
from numpy import reshape
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans

import file and shapio test

In [ ]:
df1 = pd.read_csv('') #encoding= 'unicode_escape')   #讀取資料 import your data here
df1.isnull().values  #缺失值
~df1.applymap(np.isreal).values  #非數字

from scipy.stats import shapiro
Y = df1.iloc[:, 1:].values
shapiro(Y)

correlation

In [ ]:
correlation_matrix = df1.corr(method='spearman', min_periods=1)  #以spearman的方式找相關係數，所有對角值均為1.0
print(correlation_matrix)
#correlation matrix
f, ax = plt.subplots(figsize=(16, 12))
sns.heatmap(correlation_matrix, vmax=.9, cbar=True, annot=True,square=True)  #放大圖片

PCA

In [ ]:
X = df1.iloc[:, 1:].values #去除了去除了id那一行
#print(X)

sc = StandardScaler()
X_train_std = sc.fit_transform(X)

#取最佳pca降維
pcan = PCA(n_components='mle')
X_train_pca = pcan.fit_transform(X_train_std)
print(pcan.explained_variance_ratio_) 
plt.scatter(X_train_pca[:, 0], X_train_pca[:, 1]) 
#這邊看pc，可以改成 X_train_pca[:, 0], X_train_pca[:, 2]，這樣可以看這樣可以看PC1.3的結果
plt.xlabel('PC 1')
plt.ylabel('PC 2')

plt.show()

pca9 = PCA(0.9)
X_train_pca = pca9.fit_transform(X_train_std) #到PC90%
print(pca9.explained_variance_ratio_)
print(pca9.components_)

# 將PC結果變成table

pca_out = PCA('mle').fit(X_train_std)
loadings = pca_out.components_
num_pc = pca_out.n_features_
pc_list = ["PC"+str(i) for i in list(range(1, num_pc+1))]
loadings_df = data_dum_1.iloc[:, 1:].from_dict(dict(zip(pc_list, loadings)))
loadings_df['variable'] = data_dum_1.iloc[:, 1:].columns.values
loadings_df = loadings_df.set_index('variable')
loadings_df
# output

# 畫pca向量圖
cluster.pcaplot(x=loadings[0], y=loadings[1], labels=data_dum_1.iloc[:, 1:].columns.values, 
    var1=round(pca_out.explained_variance_ratio_[0]*100, 2),
    var2=round(pca_out.explained_variance_ratio_[1]*100, 2), r = 700)
# get PC scores
pca_scores = PCA().fit_transform(X_train_std)

# get 2D biplot
cluster.biplot(cscore=pca_scores, loadings=loadings, labels=data_dum_1.iloc[:, 1:].columns.values, 
                   var1=round(pca_out.explained_variance_ratio_[0]*100, 2),
                   var2=round(pca_out.explained_variance_ratio_[1]*100, 2),arrowcolor='dimgray', colordot ='lightcoral' ,
                              valphaarrow = 1, arrowlinewidth = 0.8, r = 700) #, colorlist = target)
'''
#合併兩張圖
from skimage.io import imread

plt.figure(figsize=(20,10))

pcaplot = imread('/content/pcaplot_2d.png')
biplot = imread('/content/biplot_2d.png')

plt.subplot(121), plt.imshow(pcaplot), plt.axis('off'), plt.title('PCA plot', size=20)
plt.subplot(122), plt.imshow(biplot), plt.axis('off'), plt.title('PCA biplot', size=20)
plt.show() 
'''        

pca+k-means

In [ ]:
# 產生的資料組數 (10)
clusters = 10
# K 值的範圍 (2~10)
k_range = range(2, clusters + 1)
dx, dy = X_train_pca[:, 0],  X_train_pca[:, 1]
distortions = []
scores = []

# 記錄每種 K 值建出的 KMeans 模型的成效
for i in k_range:
    kmeans = KMeans(n_clusters=i,random_state=0).fit(X_train_pca)
    distortions.append(kmeans.inertia_) # 誤差平方和 (SSE)
    scores.append(silhouette_score(X_train_pca, kmeans.predict(X_train_pca))) # 側影係數
# 找出最大的側影係數來決定 K 值
selected_K = scores.index(max(scores)) + 2
# 重新建立 KMeans 模型並預測目標值
kmeans = KMeans(n_clusters=selected_K).fit(X_train_pca)
new_dy = kmeans.predict(X_train_pca)
# 新分組的資料中心點
centers = kmeans.cluster_centers_
plt.rcParams['font.size'] = 12
plt.figure(figsize=(12, 12))
# 原始資料分組
plt.subplot(221)
plt.title(f'Original data')
plt.scatter(X_train_pca.T[0], X_train_pca.T[1])
#plt.title(f'Original data ({clusters} groups)')
#plt.scatter(X_train_pca.T[0], X_train_pca.T[1], c=dy, cmap=plt.cm.Set1)
# 新資料分組
plt.subplot(222)
plt.title(f'KMeans={selected_K} groups')
plt.scatter(X_train_pca.T[0], X_train_pca.T[1], c=new_dy, cmap=plt.cm.Set3)
plt.scatter(centers.T[0], centers.T[1], marker='^', color='orange')
for i in range(centers.shape[0]): # 標上各分組中心點
    plt.text(centers.T[0][i], centers.T[1][i], str(i + 1),
             fontdict={'color': 'red', 'weight': 'bold', 'size': 24})
# 繪製誤差平方和圖 (手肘法)
plt.subplot(223)
plt.title('SSE (elbow method)')
plt.plot(k_range, distortions)
plt.plot(selected_K, distortions[selected_K - 2], 'go') # 最佳解
# 繪製係數圖
plt.subplot(224)
plt.title('Silhouette score')
plt.plot(k_range, scores)
plt.plot(selected_K, scores[selected_K - 2], 'go') # 最佳解
plt.tight_layout()
plt.show()


t-SNE

In [ ]:
#create a dataframe from the dataset
df = pd.DataFrame(data = X_train_pca,
                 columns = ["Component 1", 
                            "Component 2",
                            "Component 3",
                            "Component 4",
                            "Component 5",
                            "Component 6",
                            "Component 7",
                            "Component 8",
                            "Component 9"
                            ])  #這邊會隨著資料不同而有所增減

#merge this with the data_dum_1 data
data_dum_1 = pd.merge(data_dum_1,
              df,
              left_index=True,
              right_index=True,
              how = "inner")

#set the hyperparmateres for the model with
#the same as in the workshop
#try playing around with them
#how does that affect the outcome?
#自行調整參數
keep_dims = 2
lrn_rate = 'auto' #5000
#The ‘auto’ option sets the learning_rate to max(N / early_exaggeration / 4, 50)
# where N is the sample size, following [4] and [5].
prp = 40

#create the model
tsne = TSNE(n_components = keep_dims, 
            perplexity = prp, 
            random_state = 42,
            n_iter = 5000,
            n_jobs = -1)

tsne_df = data_dum_1.loc[:,"Component 1":f"Component 9"].copy()

#fit the model
X_embedded = tsne.fit_transform(tsne_df)

#create a dataframe from the dataset
df = pd.DataFrame(data = X_embedded,
                 columns = ["Dimension 1", 
                            "Dimension 2"])

#merge this with the data_dum_1 data
data_dum_1 = pd.merge(data_dum_1,
              df,
              left_index=True,
              right_index=True,
              how = "inner")

data_dum_1.columns
data_dum_1.head()

#color and plot the result, for example
'''
g = sns.jointplot(data = data_dum_1,
                 x = "Dimension 1",
                 y = "Dimension 2", 
                 hue = "Ch07")

'''

tsne+kmeans

In [ ]:
# 產生的資料組數 (10)
clusters = 10
# K 值的範圍 (2~10)
k_range = range(2, clusters + 1)
dx, dy = X_embedded[:,0], X_embedded[:,1]
distortions = []
scores = []

# 記錄每種 K 值建出的 KMeans 模型的成效，random state = 10是6組，9是2組
for i in k_range:
    kmeans = KMeans(n_clusters=i,random_state=10).fit(X_embedded)
    distortions.append(kmeans.inertia_) # 誤差平方和 (SSE)
    scores.append(silhouette_score(X_embedded, kmeans.predict(X_embedded))) # 側影係數
# 找出最大的側影係數來決定 K 值
selected_K = scores.index(max(scores)) + 2
# 重新建立 KMeans 模型並預測目標值
kmeans = KMeans(n_clusters=selected_K).fit(X_embedded)
new_dy = kmeans.predict(X_embedded)
# 新分組的資料中心點
centers = kmeans.cluster_centers_
plt.rcParams['font.size'] = 12
plt.figure(figsize=(12, 12))
# 原始資料分組
plt.subplot(221)
plt.title(f'Original data')
plt.scatter(X_embedded[:,0], X_embedded[:,1]) #X_embedded.T[1], , c=dy, cmap=plt.cm.Set1
#to see the 10 group, please copy under:
# plt.title(f'Original data={clusters} groups')
# plt.scatter(X_embedded.T[:,0], X_embedded.T[:,1], c=dy, cmap=plt.cm.Set1) 
# 新資料分組
plt.subplot(222)
plt.title(f'KMeans={selected_K} groups')
plt.scatter(X_embedded.T[0], X_embedded.T[1], c=new_dy, cmap=plt.cm.Set3)
plt.scatter(centers.T[0], centers.T[1], marker='^', color='orange')
for i in range(centers.shape[0]): # 標上各分組中心點
    plt.text(centers.T[0][i], centers.T[1][i], str(i + 1),
             fontdict={'color': 'red', 'weight': 'bold', 'size': 24})
# 繪製誤差平方和圖 (手肘法)
plt.subplot(223)
plt.title('SSE (elbow method)')
plt.plot(k_range, distortions)
plt.plot(selected_K, distortions[selected_K - 2], 'go') # 最佳解
# 繪製係數圖
plt.subplot(224)
plt.title('Silhouette score')
plt.plot(k_range, scores)
plt.plot(selected_K, scores[selected_K - 2], 'go') # 最佳解
plt.tight_layout()
plt.show()
